In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


In [4]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your model
model_path = "./opt_collegebot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()

def chat(question, max_length=100):
    prompt = f"Question: {question} Answer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example
user_question = "When was MANIT established?"
response = chat(user_question)
print("💬 Bot Response:")
print(response)


💬 Bot Response:
Question: When was MANIT established? Answer: MANIT was established in 1960. MANIT was established in 1970. MANIT is a major player in the aviation industry. MANIT is a major player in the semiconductor industry. MANIT is a major player in the computer industry. MANIT is a major player in the aerospace industry. MANIT is a major player in the wireless industry. MANIT is a major player in the digital economy. MANIT is a major player in the
